conda install -c conda-forge liac-arff
pip install scipy

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import arff

In [18]:
# load the ARFF file
with open('Data\Training_Dataset.arff', 'r') as f:
    data, meta = arff.loadarff(f)

# convert to list of lists
data_list = data.tolist()

# convert to pandas DataFrame
df = pd.DataFrame(data_list)

# add attribute names as columns
df.columns = meta.names()

# iterate over the DataFrame and convert each value to a string
for col in df.columns:
    df[col] = df[col].apply(lambda x: x.decode() if type(x) == bytes else x)

# save the DataFrame to a new csv file
df.to_csv('Data/new_file.csv', index=False)

# split the dataframe into train and test sets
train_df, test_df = train_test_split(df, test_size=0.25, random_state=2023)

# Select the last column and assign it to df_result
result_train_df = train_df.iloc[:, -1] #all rows of hte last column

# Select the last column and assign it to df_result
result_test_df = test_df.iloc[:, -1]

# drop the 'result' column from test_df
test_df = test_df.drop('Result', axis=1)

# save the train and test sets to new csv files
train_df.to_csv('Data/train.csv', index=False)
test_df.to_csv('Data/test.csv', index=False)

# count how many records are there in DataFrame
num_rows = df.shape[0]
# count how many records are there in train DataFrame
num_rows_train = train_df.shape[0]

# count how many records are there in DataFrame
num_rows_test = test_df.shape[0]

print('Train df size is :' , num_rows_train)
print('Test df size is :' , num_rows_test)
print('Whole df size is :' ,num_rows)
print('Test df + Train df size is :' , num_rows_train + num_rows_test)

Train df size is : 8291
Test df size is : 2764
Whole df size is : 11055
Test df + Train df size is : 11055


In [21]:
# liczenie ilości próbek w każdej z klas
class_counts = df['Result'].value_counts()

# obliczenie proporcji próbek w każdej z klas
class_proportions = class_counts / class_counts.sum()

print(class_proportions)

1     0.556943
-1    0.443057
Name: Result, dtype: float64


In [9]:
# get a list of all the column names
attributes_test = test_df.columns.tolist()

# print the list of column names
print(attributes_test)

['having_IP_Address', 'URL_Length', 'Shortining_Service', 'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix', 'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length', 'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor', 'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL', 'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe', 'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank', 'Google_Index', 'Links_pointing_to_page', 'Statistical_report']


AttributeError: 'list' object has no attribute 'length'

In [10]:
# get a list of all the column names
attributes_train = train_df.columns.tolist()

# print the list of column names
print(attributes_train)

['having_IP_Address', 'URL_Length', 'Shortining_Service', 'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix', 'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length', 'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor', 'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL', 'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe', 'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank', 'Google_Index', 'Links_pointing_to_page', 'Statistical_report', 'Result']


In [11]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2764 entries, 8469 to 10315
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   having_IP_Address            2764 non-null   object
 1   URL_Length                   2764 non-null   object
 2   Shortining_Service           2764 non-null   object
 3   having_At_Symbol             2764 non-null   object
 4   double_slash_redirecting     2764 non-null   object
 5   Prefix_Suffix                2764 non-null   object
 6   having_Sub_Domain            2764 non-null   object
 7   SSLfinal_State               2764 non-null   object
 8   Domain_registeration_length  2764 non-null   object
 9   Favicon                      2764 non-null   object
 10  port                         2764 non-null   object
 11  HTTPS_token                  2764 non-null   object
 12  Request_URL                  2764 non-null   object
 13  URL_of_Anchor                

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8291 entries, 9534 to 4951
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   having_IP_Address            8291 non-null   object
 1   URL_Length                   8291 non-null   object
 2   Shortining_Service           8291 non-null   object
 3   having_At_Symbol             8291 non-null   object
 4   double_slash_redirecting     8291 non-null   object
 5   Prefix_Suffix                8291 non-null   object
 6   having_Sub_Domain            8291 non-null   object
 7   SSLfinal_State               8291 non-null   object
 8   Domain_registeration_length  8291 non-null   object
 9   Favicon                      8291 non-null   object
 10  port                         8291 non-null   object
 11  HTTPS_token                  8291 non-null   object
 12  Request_URL                  8291 non-null   object
 13  URL_of_Anchor                8

In [13]:
train_df.head(10)

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
9534,1,-1,1,-1,1,-1,1,1,-1,-1,...,-1,1,1,1,1,-1,1,0,1,1
7624,1,1,1,1,1,1,-1,1,-1,1,...,1,1,-1,1,1,1,1,0,1,1
10634,-1,0,1,-1,1,-1,1,0,1,1,...,1,1,1,1,0,-1,1,1,1,-1
969,-1,1,-1,1,-1,1,1,1,-1,1,...,1,1,1,-1,1,-1,1,1,1,1
10857,1,-1,1,1,1,-1,0,1,-1,1,...,1,1,1,1,1,-1,1,0,1,1
6966,-1,1,-1,1,-1,-1,0,-1,-1,1,...,1,1,1,-1,1,1,-1,1,1,-1
4865,1,-1,1,1,1,-1,1,-1,1,1,...,1,1,1,-1,-1,-1,1,1,1,-1
6252,-1,-1,1,1,1,-1,1,0,1,1,...,1,1,-1,1,0,-1,1,1,1,-1
536,1,-1,1,1,1,-1,0,-1,-1,1,...,1,1,1,1,1,1,1,0,1,1
7312,1,0,-1,-1,1,-1,-1,1,-1,1,...,-1,1,-1,1,1,-1,1,-1,1,1


In [14]:
test_df.head(10)

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report
8469,-1,1,-1,1,-1,-1,1,1,-1,1,...,1,1,1,1,-1,-1,1,1,1,1
8650,1,-1,-1,1,-1,-1,1,-1,-1,1,...,1,1,1,1,-1,1,-1,1,1,1
8026,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,...,1,-1,1,-1,-1,-1,-1,-1,1,1
3843,1,1,1,1,1,-1,1,1,1,-1,...,1,-1,1,1,-1,1,-1,-1,0,1
7242,-1,-1,1,1,1,-1,0,-1,-1,-1,...,1,-1,-1,-1,1,1,1,1,-1,1
640,-1,-1,-1,1,-1,-1,0,-1,1,1,...,1,1,1,-1,1,-1,-1,1,0,1
5492,-1,-1,1,1,1,1,-1,-1,-1,1,...,1,1,1,-1,1,1,-1,1,1,1
10667,1,-1,1,1,1,-1,0,1,-1,1,...,1,1,1,1,1,1,-1,1,0,1
8921,1,-1,1,1,1,-1,1,1,1,1,...,1,1,1,-1,1,0,-1,1,0,1
9905,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,...,1,-1,1,1,-1,0,-1,1,1,1


In [8]:
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import LabelEncoder



# Convert the data into a format that can be used by the perceptron
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values

# Encode the categorical variables
encoder = LabelEncoder()
for i in range(X_train.shape[1]):
    if type(X_train[0, i]) is str:
        X_train[:, i] = encoder.fit_transform(X_train[:, i])

# Create a perceptron object
perceptron = Perceptron(max_iter=1000, tol=1e-3,alpha=0.05, random_state=0)

# Fit the perceptron to the training data
perceptron.fit(X_train, y_train)

Perceptron(alpha=0.05)

In [10]:
# Convert the test data into a format that can be used by the perceptron
X_test = test_df.values

# Encode the categorical variables in the test data
encoder = LabelEncoder()
for i in range(X_train.shape[1]):
    if type(X_train[0, i]) is str:
        X_train[:, i] = encoder.fit_transform(X_train[:, i])
        X_test[:, i] = encoder.transform(X_test[:, i])

# Make predictions on the test data
y_pred = perceptron.predict(X_test)

correct_predictions = (y_pred == result_test_df).sum()


print(correct_predictions , 'out of ',num_rows_test) 
print(correct_predictions / num_rows_test)

1453 out of  2764
0.5256874095513748


In [22]:
from keras.models import Sequential
from keras.layers import Dense

# Tworzenie sieci
model = Sequential()
model.add(Dense(32, input_dim=30, activation='relu')) # warstwa wejściowa, 32 neuronów, 31 wejść
model.add(Dense(16, activation='relu')) # pierwsza warstwa ukryta, 16 neuronów
model.add(Dense(8, activation='relu')) # druga warstwa ukryta, 8 neuronów
model.add(Dense(1, activation='sigmoid')) # warstwa wyjściowa, 1 neuron, funkcja aktywacji sigmoid

# Kompilacja sieci
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train,y_train, epochs =20, batch_size=32, validation_data=(X_test,y_pred))
test_loss, test_acc = model.evaluate(X_test,y_pred)
print('Test accuracy: ', test_acc)

NameError: name 'X_train' is not defined